In [ ]:
!pip install -r requirements.txt

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import re
import os

from azure.ai.textanalytics import TextAnalyticsClient, ExtractSummaryAction
from azure.core.credentials import AzureKeyCredential
from PyPDF2 import PdfReader

## Extraction

In [ ]:
page_pattern = re.compile(r'Página\s\d+\sde\s\d+')
text_pattern = re.compile(r"1\.\d")
cap_pattern = re.compile(r"\d\.\s+[A-Z]+")

In [ ]:
db = open("pdf/Bases de datos.pdf", "rb")
reader = PdfReader(db)

In [ ]:
corpus = []
for pagei in range(len(reader.pages)):
    page = reader.pages[pagei]
    text = page.extract_text()
    corpus.append(text)
db.close()

In [ ]:
corpus

['  UNIVERSIDAD DE EL SALVADOR EN LÍNEA  \n FACULTAD DE INGENIERÍA Y ARQUITECTURA  \n PROGRAMACION  III \nEste material ha sido proporcionado al estudiante en el marco de su formación a través de una carrera en línea en \nla Universidad de El Salvador.  Se han respetado los derechos de autor para su elaboración. El debido uso del mismo es \nresponsabilidad del estudiante . ALMACENAMIENTO DE DATOS\n1.INTRODUCCIÓN\nHasta el momento hemos estudiado el almacenamiento de datos; utilizando archivos de texto, \nasí como tablas contendidas en bases de datos.  \nEs importante tener en mente los conceptos relacionados al manejo de bases de datos \nrelacionales, y su gestión a través de lenguaje SQL.  \nPero de nada serviría tener una aplicación y una base de datos; sin que se relacionaran. \nRecordemos que a  travé s de los programas de nuestra aplicación, vamos a gestionar los datos \nalmacenados en l a base de datos.  \nPara efectuar dicha correlación de componentes, deberemos  lograr enlazar 

In [ ]:
corpus[4]

'  UNIVERSIDAD DE EL SALVADOR EN LÍNEA  \n                                         FACULTAD DE INGENIERÍA Y ARQUITECTURA  \n                                                                                                        PROGRAMACIONIII  \n \nEste material ha sido proporcionado al estudiante en el marco de su formación a través de una carrera en línea en \nla Universidad de El Salvador.  Se han respetado los derechos de autor para su elaboración. El debido uso del mismo es \nresponsabilidad del estudiante .  \n \n3.2 MODELO DE TRES C APAS  \nEs ideal para ambientes web. Sus características son:  \n\uf0fc Las instrucciones se envían a una capa media, entre el cliente y el servidor. Dicha capa \nintermedia envía las instrucciones SQL a la base de datos, y recoge los resultados de esta \npara exponerlos al cliente.  \n \n\uf0fc El cliente no tiene contacto directo con la base de datos.  \n \n\uf0fc El driver JDBC solamente reside en la capa media, que a su vez tiene todo el control

In [ ]:
# random_numbers = extract_text("/work/pdf/Generar numeros pseudo aleatorios.pdf")
# poo_objects = extract_text("/work/pdf/POO Clases y Objetos.pdf")
# db = extract_text("/work/pdf/Bases de datos.pdf")

In [ ]:
# random_numbers = re.split(text_pattern,random_numbers)

Due the split was in number.number pattern, at index the document was splitted multiple time, so was necessary remove those splits. From there borns the new caps variable.

## Traslation

Using the traslation functionality to traslate from english output to spanish due model gives the output in english.

## Analysis

Using language services from Azure Cognitive Services from Azure

### Authentication:

In [ ]:
key = os.environ["KEY"]
endpoint = os.environ["ENDPOINT"]

In [ ]:
text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(key),
    )

### Extraction:

In [ ]:
def sample_extractive_summarization(client, document:list[str]):

    poller = client.begin_analyze_actions(
        document,
        actions=[
            ExtractSummaryAction(max_sentence_count=4)
        ],
    )

    document_results = poller.result()
    for result in document_results:
        extract_summary_result = result[0]  # first document, first result
        if extract_summary_result.is_error:
            print("...Is an error with code '{}' and message '{}'".format(
                extract_summary_result.code, extract_summary_result.message
            ))
        else:
            output =  "".join([sentence.text for sentence in extract_summary_result.sentences])
            return output

In [ ]:
doc_summ = sample_extractive_summarization(text_analytics_client, corpus)

### Extractive:

In [ ]:
def extractive_summarization(client, doc):

    document = doc
    output = []
    summary = ''

    poller = text_analytics_client.begin_extract_summary(document)
    extract_summary_results = poller.result()
    for result in extract_summary_results:
        if result.kind == "ExtractiveSummarization":
            summary =  " ".join([sentence.text for sentence in result.sentences])
            output.append(summary)
            print("Summaries extractive:")
        elif result.is_error is True:
            print("...Is an error with code '{}' and message '{}'".format(
                result.error.code, result.error.message
            ))
    
    return output

### Abstractive:

In [ ]:
def abstractive_summarization(client,docs:list[str]):

    document:list[str] = docs
    output = []

    poller = text_analytics_client.begin_abstractive_summary(document)
    abstractive_summary_results = poller.result()
    for result in abstractive_summary_results:
        if result.kind == "AbstractiveSummarization":
            print("Summaries abstracted:")
            sumsum = [summary.text for summary in result.summaries]
            output.append(sumsum)
        elif result.is_error is True:
            print("...Is an error with code '{}' and message '{}'".format(
                result.error.code, result.error.message
            ))    
    
    return output

In [ ]:
# db_summary = extractive_summarization(text_analytics_client, db)
# random_summary = abstractive_summarization(text_analytics_client, random_numbers)

In [ ]:
# sum_sum = [page for array in random_summary for page in array]

### Persistence:

In [ ]:
def persistence(datalist:list[str], name:str) -> None:
    with open(name, mode="a", encoding='utf8') as f:
        for letter in datalist:
            f.write(letter)

In [ ]:
persistence(doc_summ, "jum.pdf")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=67cfeb36-b07e-45a6-acf2-83ce38aa6dfe' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>